In [2]:
import pandas as pd
import numpy as np

In [3]:
def data_cleaning(paths, name):
    ### COLLECT DATA RAW
    dataraw = []
    for path in paths:
        with open(path) as f:
            lines = f.readlines()
        dataraw = dataraw + lines
        f.close()
        
    print("len of datas:",len(dataraw))
        
    ### CONVERT To DataFrame
    all_feats = []
    all_dicts = []
    for l in dataraw:
        dict_ = eval(l[1:-2])
        all_dicts.append(dict_)
        for k in dict_.keys():
            if k not in all_feats:
                all_feats.append(k)
    
    data = pd.DataFrame(all_dicts,columns=all_feats)
    
    ### DATA Cleaning
    for col in list(data.columns):
        if "price" in col or "Price" in col or "Rates and Taxes" in col or "Levies" in col:
            if col == "price":
                print("Drop POA values")
                data = data[data[col]!="POA"]
            data[col] = data[col].str.lstrip("R ")
            data[col] = data[col].replace(r" ","",regex=True)
            data[col] = pd.to_numeric(data[col])
        
        elif "Size" in col or "size" in col :
            data[col] = data[col].str.rstrip(" m�")
            data[col] = data[col].str.rstrip(" m\B2")
            data[col] = data[col].replace(r" ","",regex=True)
            data[col] = data[col].replace(r"ha","",regex=True)
            data[col] = data[col].replace(r"acres","",regex=True)
            data[col] = data[col].replace(r"m²","",regex=True)
            data[col] = pd.to_numeric(data[col])

        elif "Coverage" in col :
            data[col] = data[col].str.rstrip(" %")
            data[col] = data[col].replace(r" ","",regex=True)
            data[col] = pd.to_numeric(data[col])

        elif "Height Restriction" in col:
            data[col] = data[col].str.rstrip(" m�")
            data[col] = data[col].str.rstrip(" m\B2")
            data[col] = data[col].replace(r" ","",regex=True)
            data[col] = data[col].replace(r"m²","",regex=True)
            data[col] = pd.to_numeric(data[col])

        elif "Date" in col or "date" in col:
            data[col] = pd.to_datetime(data[col])

        ### Format list values
        elif data[col].dtypes == np.object:
            if str(type(data[col][data[col].first_valid_index()])) == "<class 'list'>":
                series=[]
                for value in data[col]:
                    if str(value) != "nan":
                        value = [i.replace('km','') for i in value]
                        value = [float(i) for i in value]
                    series.append(value)
                data[col] = series
    
    #data = data.drop_duplicates()
    
    #data["index"] = data.index
    print("data dimensions:",data.shape)
    save_to = "../../Data/Originals/"+name
    data.to_csv(save_to,sep=";",index=False)

In [4]:
paths = ["../data_collect/data_property_v0.csv","../data_collect/Multi collect/data_property.csv",
         "../data_collect/Multi collect/v2/data_property_v2.csv","../data_collect/Multi collect/v3/data_property_v3.csv",
         "../data_collect/Multi collect/v4/data_property_v4.csv","../data_collect/Multi collect/v5/data_property_v2.csv",
         "../data_collect/Multi collect/v6/data_property_v6.csv","../data_collect/Multi collect/v7/data_property_v2.csv"
        ]

data_cleaning(paths,"Dataset.csv")

len of datas: 11367
Drop POA values
data dimensions: (11296, 311)
